In [1]:
%pip install ctransformers==0.2.5
%pip install sentence-transformers==2.2.2
%pip install flask
%pip install pypdf
%pip install python-dotenv
%pip install -U langchain-pinecone

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached langchain_pinecone-0.0.3-py3-none-any.whl.metadata (1.4 kB)
Using cached langchain_pinecone-0.0.3-py3-none-any.whl (8.3 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import re
import pinecone
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

/Users/rachana/experiments/End-to-end-Medical-Chatbot-using-Llama2/.conda/lib/python3.8/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "513a00d6-cd4a-4713-9286-bc58248b8c06"
PINECONE_API_ENV = "gcp-starter"
host ="https://medical-chatbot-5g67xuc.svc.gcp-starter.pinecone.io"

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
%pwd
%cd ..

/Users/rachana/experiments/End-to-end-Medical-Chatbot-using-Llama2


In [6]:
extracted_data = load_pdf("data/")

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)

In [9]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

In [12]:
import pinecone

In [13]:
from pinecone import Pinecone

In [14]:
pc = Pinecone(api_key="513a00d6-cd4a-4713-9286-bc58248b8c06")

In [15]:
index_name="medical-chatbot"

In [16]:
pc_index = pc.Index(index_name)

In [17]:
from langchain.vectorstores import Pinecone as PC

In [21]:
docs_chunks = [t.page_content for t in text_chunks]

In [23]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [25]:
pinecone_index = PC.from_texts(
    docs_chunks,
    embeddings,
    index_name=index_name
)

In [26]:
pinecone_index

In [27]:
query = "What are Allergies"

docs=pinecone_index.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [28]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [29]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [31]:
%pwd

'/Users/rachana/experiments/End-to-end-Medical-Chatbot-using-Llama2'

In [34]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-661377d8-324e0eb979b08a1c7209b7aa;9f1914f7-c6aa-4f5e-87e3-33673a996080)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.